In [1]:
import data_extraction
import pandas as pd
import yaml

pd.set_option('display.max_columns', None)

myObj = data_extraction.DataExtractor()

stream = open('db_creds.yaml', 'r') 
headers = yaml.safe_load(stream)

number = myObj.list_number_of_stores(endpoint="/prod/number_stores",header= headers)
print(number)

451


In [2]:


intNumStores = int(number)

df = myObj.retrieve_stores_data(endpoint="/prod/store_details/",header= headers,totalStores=intNumStores)

type(df)

DataFrame constructor not properly called!


pandas.core.frame.DataFrame

df is a dataframe that now contains the data from our table
These are the first & last few lines of the database then!

In [3]:
df.head(5)

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent,message
0,0.0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe,NaN
1,1.0,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe,NaN
2,2.0,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe,NaN
3,3.0,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe,NaN
4,4.0,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe,NaN


In [4]:
df.tail(5)

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent,message
446,446.0,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,None,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe,NaN
447,447.0,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,D23PCWSM6S,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL,NaN
448,448.0,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High...",51.62907,None,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe,NaN
449,449.0,"Baumplatz 6\n80114 Kötzting, Bretten",49.03685,None,Bretten,BR-662EC74C,35,2020-10-17,Local,8.70745,DE,Europe,NaN
450,450.0,"Gotthilf-Rose-Straße 7/3\n45457 Feuchtwangen, ...",50.64336,None,Bad Honnef,BA-B4AED588,36,2001-05-12,Local,7.2278,DE,eeEurope,NaN


Let's drop "lat" and "message" as they appear to be rubbish

In [5]:
df = df.drop(columns=["lat","message"])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          444 non-null    float64
 1   address        444 non-null    object 
 2   longitude      444 non-null    object 
 3   locality       444 non-null    object 
 4   store_code     444 non-null    object 
 5   staff_numbers  444 non-null    object 
 6   opening_date   444 non-null    object 
 7   store_type     444 non-null    object 
 8   latitude       443 non-null    object 
 9   country_code   444 non-null    object 
 10  continent      444 non-null    object 
dtypes: float64(1), object(10)
memory usage: 38.9+ KB


In [7]:
df.dtypes

index            float64
address           object
longitude         object
locality          object
store_code        object
staff_numbers     object
opening_date      object
store_type        object
latitude          object
country_code      object
continent         object
dtype: object

index seems to be an integer

In [8]:
df['index'] = pd.to_numeric(df['index'],errors="coerce")

As do latitude and longitude

In [9]:
df['longitude']= pd.to_numeric(df['longitude'],errors="coerce")
df['latitude'] = pd.to_numeric(df['latitude'],errors="coerce")

lets drop the rows where the lat and long failed 

In [10]:
df.dropna(inplace=True,axis="index",how="any") # drops the whole row if any columns have an nan

We have some date columns

In [11]:
df['opening_date']=pd.to_datetime(df['opening_date'],infer_datetime_format=True, errors='coerce')

inspect some potentially categorical data:
Country
Country Code

In [12]:
df["continent"].describe()

count        433
unique         4
top       Europe
freq         376
Name: continent, dtype: object

In [13]:
df["country_code"].describe()

count     433
unique      3
top        GB
freq      260
Name: country_code, dtype: object

Not bad, though there's 4 continents and 3 codes!

In [14]:
df["continent"] = df["continent"].astype("category")
df["country_code"] = df["country_code"].astype("category")

Store Types as categories?

In [15]:
df["store_type"].describe()

count       433
unique        4
top       Local
freq        251
Name: store_type, dtype: object

Yes. What about duplicate lines?

In [16]:
df["store_type"] = df["store_type"].astype("category")

df.duplicated().sum()

0

None! ok. Let's just check on the critical columns only - not system generated ones

In [17]:
df.duplicated(subset=["address", "locality"],keep="first").sum()

0

Still none

clean up address

In [18]:
df["address"] = df["address"].str.replace('\W', ' ', regex=True)

Now put the code into a local database maybe

In [19]:
import database_utils


myObject = database_utils.DatabaseConnector()


myObject.upload_to_db(df,"dim_store_details")

Ideas for future:
Misspellings: https://medium.com/geekculture/how-to-fix-misspelled-words-for-your-next-nlp-project-with-one-line-of-code-98888a5ff207
Common variants of companies - "ltd." vs "ltd" vs "limited"